In [74]:
from morpholog import Morpholog 
import re
from collections import Counter, defaultdict
import pickle
import json

with open(r"train_data\train.txt", "r", encoding="utf-8") as file:
    text = file.read()

In [75]:
class Morphology_Tok:
    def __init__(self, text):
        self.text = text

    def tokenize_with_morpholog(self):
            analyzer = Morpholog()
            roots = []
            endings = []

            for word in self.text:
                root = analyzer.get_roots(word)
                ending = analyzer.get_ending(word)

                if root:
                    cleaned_root = root[0].strip("',\"().")
                    roots.append(cleaned_root)

                if ending:
                    cleaned_ending = ending.strip("',\"().+")
                    endings.append(cleaned_ending)

            return roots, endings
    
# morph = Morphology_Tok(text)
# roots, endings = morph.tokenize_with_morpholog()
# for root in roots:
#      print(root)


In [76]:
class PairFinder:
    def __init__(self, threshold):
        self.threshold = threshold
        self.pair_counts = {}
    
    def find_and_return_pairs(self, text):
        # Инициализирует словарь для хранения парных частот
        for i in range(len(text) - 1):
            pair = text[i:i+2]
            if all(c.isalpha() for c in pair):  # Рассматриваем только алфавитные пары
                self.pair_counts[pair] = self.pair_counts.get(pair, 0) + 1
        
        # Создайте набор часто встречающихся пар
        frequent_pairs = {pair for pair, count in self.pair_counts.items() if count >= self.threshold}
        
        # Возвращайте только часто встречающиеся пары
        return frequent_pairs

# # Пример использования
# text = "Привет, это пример текста с некоторыми парами, такими как 'ab', 'cd' и 'ef'."
# pair_finder = PairFinder(threshold=3)
# result = pair_finder.find_and_return_pairs(text)
# print("Часто встречающиеся пары:", result)


In [77]:
class Character_Tok:
    def __init__(self, text):
        self.unique_characters = set()
        self.text = text

    def collect_unique_characters(self):
        for word in self.text:
            self.unique_characters.update(set(word))

    def get_unique_characters(self):
        return self.unique_characters
    
# collector = Character_Tok(text)
# collector.collect_unique_characters()
# unique_characters = collector.get_unique_characters()
# print(unique_characters)

In [78]:
class CustomTokenizer:
    def __init__(self, text):
        self.text = text
        self.morphology_tok = Morphology_Tok(text)
        self.character_tok = Character_Tok(text)
        self.pair_finder = PairFinder(threshold=3)
        self.symbols = ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ё', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р',
                        'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я',
                        'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',']
        
        # Объединяем все токены в один словарь
        roots, endings = self.morphology_tok.tokenize_with_morpholog()
        frequent_pairs = self.pair_finder.find_and_return_pairs(self.text)
        unique_characters = self.character_tok.get_unique_characters()
        self.all_tokens = roots + endings + list(frequent_pairs) + list(unique_characters) + self.symbols


        print(f"Корни: {roots}")
        print(f"Окончания: {endings}")
        print(f"Пары: {frequent_pairs}")
        print(f"Символы: {unique_characters}")
        print(f"Символы: {self.symbols}")

        # Присваиваем значения токенам
        self.token_values = {token: i for i, token in enumerate(self.all_tokens)}

    def encode_token(self, token):
        return [self.token_values[token]]  # Получаем числовое значение токена
    
    def decode_token(self, encoded_value):
        return self.all_tokens[encoded_value[0]]   # Преобразуем числовое значение обратно в строку
    
    def tokenize(self):
        encoded_tokens = [self.encode_token(token) for token in self.all_tokens]
        return encoded_tokens
    
    def save_to_pickle(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.token_values, f)
        print(f"Токенизатор сохранен в файл {filename} в формате pkl.")
    
    def save_to_json(self, filename):
        with open(filename, 'w',  encoding='utf-8') as f:
            json.dump(self.token_values, f, ensure_ascii=False, indent=4)
        print(f"Токенизатор сохранен в файл {filename} в формате json.")


with open(r"train_data\train.txt", "r", encoding="utf-8") as file:
    text = file.read()

#Обучение токенизатора
tokenizer = CustomTokenizer(text)
tokenizer.save_to_pickle(r"gpt\tokenizer.pkl")
tokenizer.save_to_json(r"gpt\tokenizer.json")

Корни: ['р', 'с', 'с', 'к', 'з', 'о', 'к', 'дал', 'л', 'х', '\n', '\n', 'i', '\n', 'л', 'е', 'к', 'с', 'е', 'я', 'и', 'к', 'о', 'в', 'з', 'к', 'о', 'в', 'л', 'и', 'в', 'с', 'так', 'р', 'ы', 'е', '', 'дал', 'о', 'б', 'л', 'е', 'с', 'к', 'о', 'так', 'шир', 'л', 'и', 'ф', 'о', 'в', 'в', 'шир', 'и', 'е', 'с', 'я', 'ч', 'ь', 'и', 'х', 'так', 'о', 'о', 'г', 'х', '', 'к', 'дал', 'л', 'ы', '', 'в', 'ы', 'к', 'о', 'в', 'ы', 'о', 'и', 'б', 'ы', 'л', 'и', 'дал', 'в', 'о', '', 'л', 'е', 'к', 'с', 'е', 'й', 'у', 'з', 'л', 'о', 'б', 'э', 'так', 'о', 'м', 'в', 'с', 'и', 'б', 'и', 'р', 'и', '', 'в', 'о', 'дал', 'в', 'о', 'р', 'е', 'к', 'так', 'о', 'р', 'ж', 'о', 'й', 'так', 'ю', 'р', 'ь', 'м', 'ы', 'е', 'г', 'о', 'о', 'с', 'так', 'о', 'в', 'и', 'л', 'с', 'так', 'р', 'и', 'к', '', 'к', 'л', 'о', 'и', 'л', 'с', 'я', '', 'о', 'щ', 'у', 'п', 'л', 'к', 'дал', 'л', 'ы', 'и', 'в', 'о', 'с', 'к', 'л', 'и', 'к', 'у', 'л', ':', '\n', '–', 'э', 'х', '!', 'шир', 'и', 'в', 'л', 'я', 'и', 'х', '', 'шир', 'и', 'в', 

In [79]:
# class CombinedTokenizer:
#     def __init__(self, text, num_merges):
#         self.text = text
#         self.num_merges = num_merges
#         self.combined_vocab = None  # Инициализация атрибута

#     def tokenize(self):
#         # Инициализация и обработка текста морфологическим токенизатором
#         morphology_tok = Morphology_Tok(self.text)
#         roots, endings = morphology_tok.tokenize_with_morpholog()

#         character_tok = Character_Tok(self.text)
#         character_tok.collect_unique_characters()
#         chars = character_tok.get_unique_characters()

#         print(f"Корни: {roots}")
#         print(f"Окончания: {endings}")
#         print(f"BPE словарь: {list(bpe_tok.vocab.keys())}")
#         print(f"Символы: {list(chars)}")

#         # Объединение результатов в один словарь
#         self.combined_vocab = set(roots + endings + list(bpe_tok.vocab.keys()) + list(chars))

#         self.token_to_id = {token: id for id, token in enumerate(self.combined_vocab)}
#         self.id_to_token = {id: token for token, id in self.token_to_id.items()}

#         # Преобразование токенов в числовые идентификаторы
#         token_ids = [self.token_to_id[token] for token in self.combined_vocab]
#         return token_ids
    
#     def save_vocab(self):
#         # Сохранение словаря в файл
#         with open(r'gpt/token_to_id.pkl', 'wb') as file:
#             pickle.dump(self.token_to_id, file)
#         print("Словарь успешно сохранен.")
        
#         # Сохранение обратного словаря в файл
#         with open(r'gpt/id_to_token.pkl', 'wb') as file:
#             pickle.dump(self.id_to_token, file)
#         print("Обратный словарь успешно сохранен.")

#     def save_vocab_json(self):
#         # Сохранение словаря token_to_id в файл JSON
#         with open(r'gpt/token_to_id.json', 'w', encoding='utf-8') as file:
#             json.dump(self.token_to_id, file, ensure_ascii=False, indent=4)
#         print("Словарь token_to_id успешно сохранен в формате JSON.")

#         # Сохранение обратного словаря id_to_token в файл JSON
#         with open(r'gpt/id_to_token.json', 'w', encoding='utf-8') as file:
#             json.dump(self.id_to_token, file, ensure_ascii=False, indent=4)
#         print("Обратный словарь id_to_token успешно сохранен в формате JSON.")
    
# def test_tokenizer(tokenizer):
#     tokens = tokenizer.tokenize()
#     print(f"Токены: {tokens}")
#     print(f"Количество токенов: {len(tokens)}")

# # Пример использования
# text = "Пример текста для токенизации"
# num_merges = 100  # Количество итераций для BPE
# tokenizer = CombinedTokenizer(text, num_merges)
# token_ids = tokenizer.tokenize()

# # Сохранение словарей
# tokenizer.save_vocab()
# tokenizer.save_vocab_json()

# # Сохранение экземпляра токенизатора в файл
# with open(r'gpt/tokenizer.pkl', 'wb') as file:
#     pickle.dump(tokenizer, file)
# print("Токенизатор успешно сохранен.")

In [80]:

class SavedTokenizerTester:
    def __init__(self, pickle_filename, json_filename):
        self.pickle_filename = pickle_filename
        self.json_filename = json_filename

    def load_tokenizer_from_pickle(self):
        with open(self.pickle_filename, 'rb') as f:
            token_values = pickle.load(f)
        return token_values

    def load_tokenizer_from_json(self):
        with open(self.json_filename, 'r', encoding='utf-8') as f:
            token_values = json.load(f)
        return token_values

    def test_saved_tokenizers(self, original_text):
        saved_pickle_tokenizer = self.load_tokenizer_from_pickle()
        saved_json_tokenizer = self.load_tokenizer_from_json()

        # Encode and decode tokens using both saved tokenizers
        encoded_tokens_pickle = [saved_pickle_tokenizer.get(token, -1) for token in original_text.split()]
        decoded_tokens_pickle = [saved_pickle_tokenizer.get(encoded, "UNKNOWN") for encoded in encoded_tokens_pickle]

        encoded_tokens_json = [saved_json_tokenizer.get(token, -1) for token in original_text.split()]
        decoded_tokens_json = [saved_json_tokenizer.get(encoded, "UNKNOWN") for encoded in encoded_tokens_json]

        # Print results
        print("Original Text:", original_text)
        print("Encoded Tokens (Pickle):", encoded_tokens_pickle)
        print("Decoded Tokens (Pickle):", decoded_tokens_pickle)
        print("Encoded Tokens (JSON):", encoded_tokens_json)
        print("Decoded Tokens (JSON):", decoded_tokens_json)

# Example usage
original_text = "Привет, мир! Это тестовый текст."
tester = SavedTokenizerTester(pickle_filename="gpt/tokenizer.pkl", json_filename="gpt/tokenizer.json")
tester.test_saved_tokenizers(original_text)


Original Text: Привет, мир! Это тестовый текст.
Encoded Tokens (Pickle): [-1, -1, -1, -1, -1]
Decoded Tokens (Pickle): ['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN']
Encoded Tokens (JSON): [-1, -1, -1, -1, -1]
Decoded Tokens (JSON): ['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN']
